## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

#client.list_experiments()
client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/mlruns/1', creation_time=1681830312654, experiment_id='1', last_update_time=1681830312654, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [4]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 8e4cc14ed4a54858a903f6d95168b7d2, rmse: 6.2876
run id: f09315592d984fee9e8e0e288498d5e6, rmse: 6.7423
run id: db149f01de0e42efb88f20584a87ff9b, rmse: 6.9128
run id: 69235c77b7cf4e4e8e8ed9494319e976, rmse: 6.9231


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [7]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [8]:
run_id = "db149f01de0e42efb88f20584a87ff9b"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

2023/04/21 11:26:55 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/21 11:26:55 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/04/21 11:26:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1682076415653, current_stage='None', description=None, last_updated_timestamp=1682076415653, name='nyc-taxi-regressor', run_id='db149f01de0e42efb88f20584a87ff9b', run_link=None, source='/home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/mlruns/1/db149f01de0e42efb88f20584a87ff9b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [9]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


In [10]:
model_version = 3
new_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1682076415653, current_stage='Staging', description=None, last_updated_timestamp=1682076899311, name='nyc-taxi-regressor', run_id='db149f01de0e42efb88f20584a87ff9b', run_link=None, source='/home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/mlruns/1/db149f01de0e42efb88f20584a87ff9b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [11]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1682076415653, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-04-21', last_updated_timestamp=1682077080846, name='nyc-taxi-regressor', run_id='db149f01de0e42efb88f20584a87ff9b', run_link=None, source='/home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/mlruns/1/db149f01de0e42efb88f20584a87ff9b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [12]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [13]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [14]:
#client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


#run_id identifies the model version (in this case version 3)
mlflow.artifacts.download_artifacts(run_id=run_id, artifact_path='preprocessor', dst_path='.')

'/home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/preprocessor'

In [15]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [16]:
X_test = preprocess(df, dv)

In [17]:
target = "duration"
y_test = df[target].values

In [18]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 288 ms, sys: 34.7 ms, total: 322 ms
Wall time: 532 ms


{'rmse': 6.659623830022514}

In [19]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 8.34 s, sys: 793 ms, total: 9.13 s
Wall time: 10.9 s


{'rmse': 6.887271811411502}

In [21]:
#just for demo, even if the actual staging model have worst rmse and takes more time to process
#we will transition it to production, and at the same time we archive the previous production model

client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1682076415653, current_stage='Production', description='The model version 3 was transitioned to Staging on 2023-04-21', last_updated_timestamp=1682081799832, name='nyc-taxi-regressor', run_id='db149f01de0e42efb88f20584a87ff9b', run_link=None, source='/home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/mlruns/1/db149f01de0e42efb88f20584a87ff9b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>